In [ ]:
cd /content/drive/My Drive/zsdata

/content/drive/My Drive/zsdata


In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('dataset/train_file.csv')
df_test = pd.read_csv('dataset/test_file.csv')

In [ ]:
df.drop(['IDLink','PublishDate','Source'],axis=1,inplace=True)

In [ ]:
import re
import nltk
import inflect 
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from string import punctuation
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    new_string = [] 
    p = inflect.engine() 
    for word in text: 
        # if word is a digit, convert the digit 
        # to numbers and append into the new_string list 
        if word.isdigit(): 
            temp = p.number_to_words(word) 
            new_string.append(temp) 
  
        # append the word as it is 
        else: 
            new_string.append(word)
    text = new_string
    # Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r"\-", " ", text)
    text = re.sub(r"\=", " ", text)
    text = re.sub(r"'", " ", text)
    # text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(' +', ' ', text) 
    
    # text = re.sub(r"\s{2,}", " ", text)
    # Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    # lemmatizer=WordNetLemmatizer()
    # text=word_tokenize(text)
    # lemma = [lemmatizer.lemmatize(word) for word in text]
    # text = " ".join(lemma)
    return text

In [ ]:
df['Headline']=df['Headline'].apply(clean_text)
df['Title']=df['Title'].apply(clean_text)

In [ ]:
title_sent = df['Title'].tolist()
head_sent = df['Headline'].tolist()
senti_title = df['SentimentTitle'].tolist()
senti_head = df['SentimentHeadline'].tolist()

In [ ]:
df_test['Headline']=df_test['Headline'].apply(clean_text)
df_test['Title']=df_test['Title'].apply(clean_text)

In [ ]:
title_sent_test = df_test['Title'].tolist()
head_sent_test = df_test['Headline'].tolist()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
title_texts_train=df['Title'].astype(str)
title_texts_test=df_test['Title'].astype(str)

In [ ]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=5000)

In [ ]:
full_tfidf_word_title=word_vectorizer.fit_transform(df['Title'].values.tolist() + df_test['Title'].values.tolist())
train_tfidf_word_title = word_vectorizer.transform(df['Title'].values.tolist())
test_tfidf_word_title = word_vectorizer.transform(df_test['Title'].values.tolist())

In [ ]:
full_tfidf_word_head=word_vectorizer.fit_transform(df['Headline'].values.tolist() + df_test['Headline'].values.tolist())
train_tfidf_word_head = word_vectorizer.transform(df['Headline'].values.tolist())
test_tfidf_word_head = word_vectorizer.transform(df_test['Headline'].values.tolist())

In [ ]:
df=df.reset_index()
df_test=df_test.reset_index()

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
n_comp_title = 10
svd_obj_title = TruncatedSVD(n_components=n_comp_title, algorithm='arpack')
svd_obj_title.fit(full_tfidf_word_title)
train_svd_word_title = pd.DataFrame(svd_obj_title.transform(train_tfidf_word_title))
test_svd_word_title = pd.DataFrame(svd_obj_title.transform(test_tfidf_word_title))
    
train_svd_word_title.columns = ['svd_word_title_'+str(i) for i in range(n_comp_title)]
test_svd_word_title.columns = ['svd_word_title_'+str(i) for i in range(n_comp_title)]
df = pd.concat([df, train_svd_word_title], axis=1)
df_test = pd.concat([df_test, test_svd_word_title], axis=1)

In [ ]:
n_comp_head = 20
svd_obj_head = TruncatedSVD(n_components=n_comp_head, algorithm='arpack')
svd_obj_head.fit(full_tfidf_word_head)
train_svd_word_head = pd.DataFrame(svd_obj_head.transform(train_tfidf_word_head))
test_svd_word_head = pd.DataFrame(svd_obj_head.transform(test_tfidf_word_head))
    
train_svd_word_head.columns = ['svd_word_head_'+str(i) for i in range(n_comp_head)]
test_svd_word_head.columns = ['svd_word_head_'+str(i) for i in range(n_comp_head)]
df = pd.concat([df, train_svd_word_head], axis=1)
df_test = pd.concat([df_test, test_svd_word_head], axis=1)

In [ ]:
# df.head()

,index,Title,Headline,Topic,Facebook,GooglePlus,LinkedIn,SentimentTitle,SentimentHeadline,svd_word_title_0,svd_word_title_1,svd_word_title_2,svd_word_title_3,svd_word_title_4,svd_word_title_5,svd_word_title_6,svd_word_title_7,svd_word_title_8,svd_word_title_9,svd_word_head_0,svd_word_head_1,svd_word_head_2,svd_word_head_3,svd_word_head_4,svd_word_head_5,svd_word_head_6,svd_word_head_7,svd_word_head_8,svd_word_head_9,svd_word_head_10,svd_word_head_11,svd_word_head_12,svd_word_head_13,svd_word_head_14,svd_word_head_15,svd_word_head_16,svd_word_head_17,svd_word_head_18,svd_word_head_19
0,0,obama lay wreath arlington nation cemeteri,obama lay wreath arlington nation cemeteri pre...,obama,-1,-1,-1,0.000000,-0.053300,0.069942,0.098037,-0.030366,-0.009397,0.004577,-0.003603,-0.026209,-0.012369,-0.014348,-0.009552,0.127130,0.119910,-0.014466,-0.035393,-0.015969,0.002346,-0.003407,-0.003056,-0.030564,0.004308,-0.007657,0.003632,0.005470,-0.005866,0.029872,-0.039520,-0.025583,-0.028987,-0.028990,0.021584
1,1,look health chines economi,tim haywood invest director busi unit head fix...,economy,-1,-1,-1,0.208333,-0.156386,0.170616,-0.126744,-0.044926,-0.031687,-0.017804,-0.044893,-0.020202,0.008988,0.001626,-0.021411,0.113929,-0.078590,0.076412,-0.009213,0.036017,-0.110349,0.074673,-0.019842,-0.102723,-0.000174,0.126065,-0.082346,0.048653,0.014303,-0.063545,0.009749,-0.041140,-0.063500,-0.018073,-0.030983
2,2,nouriel roubini global economi back two thousa...,nouriel roubini nyu professor chairman roubini...,economy,-1,-1,-1,-0.425210,0.139754,0.367370,-0.281357,-0.069716,0.261570,0.315182,-0.015521,-0.009713,-0.016377,0.015314,-0.021042,0.096368,-0.108906,0.142665,-0.002604,-0.007570,-0.187178,-0.084442,-0.002412,0.030397,-0.001891,0.017325,0.060917,0.047057,0.016235,-0.003469,-0.002654,-0.081524,-0.023371,0.025362,-0.023478
3,3,finland gdp expand,finland economi expand margin three month end ...,economy,-1,-1,-1,0.000000,0.026064,0.019095,-0.010422,0.002451,0.002700,0.007320,0.020546,-0.005672,0.007810,-0.009305,-0.009812,0.083676,-0.093481,0.090921,-0.068185,-0.030124,0.127849,0.041482,-0.009036,-0.024856,-0.026975,0.007773,-0.023461,-0.076033,-0.059445,-0.001889,-0.026110,-0.011968,-0.003300,0.005955,0.035306
4,4,tourism govt spend buoy thai economi januari,tourism public spend continu boost economi jan...,economy,-1,-1,-1,0.000000,0.141084,0.089519,-0.076874,-0.030822,-0.013233,-0.006965,-0.026033,-0.006621,0.002964,-0.002988,-0.012661,0.092500,-0.105984,0.099638,-0.043557,-0.004375,0.007083,0.050274,-0.005328,0.018547,-0.017792,-0.010811,-0.021915,-0.037837,-0.027574,-0.015055,0.013571,-0.017623,-0.015290,-0.028436,0.058854


In [ ]:
# train_y = pd.concat([df['SentimentTitle'], df['SentimentHeadline']], axis=1)
train_y_t = df['SentimentTitle']
train_y_h = df['SentimentHeadline']

In [ ]:
train=df.drop(['index','Title','Headline','Facebook','GooglePlus','LinkedIn','SentimentTitle','SentimentHeadline'],axis=1)
# test=df_test.drop(['IDLink','PublishDate','Source','index','Title','Headline'],axis=1)

In [ ]:
# train.head()

,Topic,svd_word_title_0,svd_word_title_1,svd_word_title_2,svd_word_title_3,svd_word_title_4,svd_word_title_5,svd_word_title_6,svd_word_title_7,svd_word_title_8,svd_word_title_9,svd_word_head_0,svd_word_head_1,svd_word_head_2,svd_word_head_3,svd_word_head_4,svd_word_head_5,svd_word_head_6,svd_word_head_7,svd_word_head_8,svd_word_head_9,svd_word_head_10,svd_word_head_11,svd_word_head_12,svd_word_head_13,svd_word_head_14,svd_word_head_15,svd_word_head_16,svd_word_head_17,svd_word_head_18,svd_word_head_19
0,obama,0.069942,0.098037,-0.030366,-0.009397,0.004577,-0.003603,-0.026209,-0.012369,-0.014348,-0.009552,0.127130,0.119910,-0.014466,-0.035393,-0.015969,0.002346,-0.003407,-0.003056,-0.030564,0.004308,-0.007657,0.003632,0.005470,-0.005866,0.029872,-0.039520,-0.025583,-0.028987,-0.028990,0.021584
1,economy,0.170616,-0.126744,-0.044926,-0.031687,-0.017804,-0.044893,-0.020202,0.008988,0.001626,-0.021411,0.113929,-0.078590,0.076412,-0.009213,0.036017,-0.110349,0.074673,-0.019842,-0.102723,-0.000174,0.126065,-0.082346,0.048653,0.014303,-0.063545,0.009749,-0.041140,-0.063500,-0.018073,-0.030983
2,economy,0.367370,-0.281357,-0.069716,0.261570,0.315182,-0.015521,-0.009713,-0.016377,0.015314,-0.021042,0.096368,-0.108906,0.142665,-0.002604,-0.007570,-0.187178,-0.084442,-0.002412,0.030397,-0.001891,0.017325,0.060917,0.047057,0.016235,-0.003469,-0.002654,-0.081524,-0.023371,0.025362,-0.023478
3,economy,0.019095,-0.010422,0.002451,0.002700,0.007320,0.020546,-0.005672,0.007810,-0.009305,-0.009812,0.083676,-0.093481,0.090921,-0.068185,-0.030124,0.127849,0.041482,-0.009036,-0.024856,-0.026975,0.007773,-0.023461,-0.076033,-0.059445,-0.001889,-0.026110,-0.011968,-0.003300,0.005955,0.035306
4,economy,0.089519,-0.076874,-0.030822,-0.013233,-0.006965,-0.026033,-0.006621,0.002964,-0.002988,-0.012661,0.092500,-0.105984,0.099638,-0.043557,-0.004375,0.007083,0.050274,-0.005328,0.018547,-0.017792,-0.010811,-0.021915,-0.037837,-0.027574,-0.015055,0.013571,-0.017623,-0.015290,-0.028436,0.058854


In [ ]:
test=df_test.drop(['IDLink','PublishDate','Source','index','Title','Headline','Facebook','GooglePlus','LinkedIn'],axis=1)

In [ ]:
# test.head()

,Topic,svd_word_title_0,svd_word_title_1,svd_word_title_2,svd_word_title_3,svd_word_title_4,svd_word_title_5,svd_word_title_6,svd_word_title_7,svd_word_title_8,svd_word_title_9,svd_word_head_0,svd_word_head_1,svd_word_head_2,svd_word_head_3,svd_word_head_4,svd_word_head_5,svd_word_head_6,svd_word_head_7,svd_word_head_8,svd_word_head_9,svd_word_head_10,svd_word_head_11,svd_word_head_12,svd_word_head_13,svd_word_head_14,svd_word_head_15,svd_word_head_16,svd_word_head_17,svd_word_head_18,svd_word_head_19
0,economy,0.138004,-0.090521,-0.045574,-0.024249,-0.018675,-0.042398,-0.025223,-0.014266,-0.008969,-0.014172,0.132733,0.018558,0.017279,-0.037892,0.074507,0.033010,-0.036353,0.059464,0.049943,0.006739,0.045617,-0.008086,0.154999,-0.136427,0.066305,0.027538,0.058098,-0.047171,-0.105594,0.088549
1,microsoft,0.063667,0.014620,0.180104,-0.024038,-0.014908,-0.006613,0.009710,0.003978,-0.026557,-0.018969,0.074832,-0.084285,-0.119092,0.007166,0.004405,-0.029927,0.001396,0.011897,-0.002100,-0.053261,-0.017382,-0.002433,0.001236,0.035690,-0.036030,-0.010555,0.011849,-0.021235,-0.025532,0.049646
2,microsoft,0.069247,0.032162,0.097001,-0.016555,-0.013294,0.002569,0.126644,-0.042953,0.104863,-0.097136,0.073418,-0.076882,-0.094991,-0.006781,0.002506,-0.014278,0.011894,-0.005779,0.000091,-0.031533,0.005102,0.012087,-0.007885,0.013034,-0.014281,-0.012878,-0.005534,0.030030,-0.019967,0.041294
3,economy,0.123790,-0.091149,-0.040126,-0.004177,-0.031025,0.003302,0.016274,-0.017067,-0.013017,0.007944,0.071796,-0.067320,0.073395,-0.005986,-0.004287,-0.061087,0.060682,-0.016070,0.018888,-0.010273,0.037099,0.038073,0.027270,0.017044,0.037175,-0.015079,-0.050069,-0.004327,0.014791,-0.014809
4,economy,0.094539,-0.061761,-0.026470,-0.013126,-0.005374,-0.012780,-0.003877,-0.002145,-0.001710,-0.007807,0.083406,-0.073622,0.060712,-0.020871,-0.009590,-0.029090,0.055459,-0.005946,0.017897,0.011120,0.004281,0.063907,0.034999,0.026925,-0.023028,-0.002529,-0.025514,-0.011251,0.017434,0.003892


In [ ]:
topic_map = {'obama':1, 'economy':2,'microsoft':3,'palestine':4}

train['Topic'] = train['Topic'].map(topic_map)
test['Topic'] = test['Topic'].map(topic_map)

In [ ]:
import lightgbm as lgb

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
model_t = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression', num_leaves=1200,
                    learning_rate=0.05, n_estimators=1000, max_depth=12,
                    metric='rmse', bagging_fraction=0.8, feature_fraction=0.8, reg_lambda=0.9)

In [ ]:
parameters = {'n_estimators':[1000,1200],'learning_rate':[0.1,0.05,0.01]}

In [ ]:
clf_t = GridSearchCV(model_t, parameters)

In [ ]:
clf_t.fit(train,train_y_t)

GridSearchCV(cv=None, error_score=nan,
             estimator=LGBMRegressor(bagging_fraction=0.8, boosting_type='gbdt',
                                     class_weight=None, colsample_bytree=1.0,
                                     feature_fraction=0.8,
                                     importance_type='split',
                                     learning_rate=0.05, max_depth=12,
                                     metric='rmse', min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=1000, n_jobs=-1,
                                     num_leaves=1200, objective='regression',
                                     random_state=None, reg_alpha=0.0,
                                     reg_lambda=0.9, silent=True, subsample=1.0,
                                     subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=N

In [ ]:
# model_t.fit(train,train_y_t)

LGBMRegressor(bagging_fraction=0.8, boosting_type='gbdt', class_weight=None,
              colsample_bytree=1.0, feature_fraction=0.8,
              importance_type='split', learning_rate=0.05, max_depth=12,
              metric='rmse', min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=1000, n_jobs=-1, num_leaves=1200,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.9, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [ ]:
model_h = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression', num_leaves=1200,
                    learning_rate=0.05, n_estimators=1000, max_depth=12,
                    metric='rmse', bagging_fraction=0.8, feature_fraction=0.8, reg_lambda=0.9)

In [ ]:
clf_h = GridSearchCV(model_h, parameters)

In [ ]:
# model_h.fit(train,train_y_h)
clf_h.fit(train,train_y_t)

GridSearchCV(cv=None, error_score=nan,
             estimator=LGBMRegressor(bagging_fraction=0.8, boosting_type='gbdt',
                                     class_weight=None, colsample_bytree=1.0,
                                     feature_fraction=0.8,
                                     importance_type='split',
                                     learning_rate=0.05, max_depth=12,
                                     metric='rmse', min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=1000, n_jobs=-1,
                                     num_leaves=1200, objective='regression',
                                     random_state=None, reg_alpha=0.0,
                                     reg_lambda=0.9, silent=True, subsample=1.0,
                                     subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=N

In [ ]:
# pred_t=model_t.predict(test)
pred_t=clf_t.predict(test)

In [ ]:
pred_h=clf_h.predict(test)

In [ ]:
testtemp = pd.read_csv('dataset/test_file.csv')

In [ ]:
# testtemp.head()

,IDLink,Title,Headline,Source,Topic,PublishDate,Facebook,GooglePlus,LinkedIn
0,tFrqIR6Chj,Sliding Economy: FG fights back with N3trn TSA...,With the 2016 budget now passed by the Nationa...,BusinessDay,economy,2016-03-29 01:41:12,0,0,1
1,DVAaGErjlF,Microsoft shows how HoloLens can bring distant...,A recent Microsoft Research video shows how th...,Daily Mail,microsoft,2016-03-29 01:41:27,121,2,13
2,OT9UIZm5M2,"Microsoft’s Twitter Robot Praises Hitler, Trum...",* Microsoft teamed with Bing to create TayTwee...,EURweb,microsoft,2016-03-29 01:47:00,12,1,0
3,lflGp3q2Fj,Flood of Central Bank Moves Can't Get World Ec...,Central bankers have managed to steer the worl...,Bloomberg via Yahoo! Finance,economy,2016-03-29 02:00:00,0,0,3
4,zDYG0SoovZ,USD/JPY: bears lining up on mixed U.S. economy...,"However, this streak of seven-day gains might ...",FXStreet,economy,2016-03-29 02:01:07,3,0,0


In [ ]:
submissions=pd.DataFrame(columns=['IDLink','SentimentTitle','SentimentHeadline'])
submissions['IDLink'] = testtemp['IDLink']
submissions['SentimentTitle'] = pred_t
submissions['SentimentHeadline'] = pred_h

In [ ]:
submissions.to_csv('submission5.csv',index=False)

In [ ]:
# sub = pd.read_csv('submission2.csv')

In [ ]:
# sub.head()

,IDLink,SentimentTitle,SentimentHeadline
0,tFrqIR6Chj,-0.010385,-0.025498
1,DVAaGErjlF,0.011471,0.034969
2,OT9UIZm5M2,-0.055839,-0.035383
3,lflGp3q2Fj,-0.005483,-0.037457
4,zDYG0SoovZ,-0.038516,-0.075608


In [ ]:
# suniv = pd.read_csv('submission3.csv')

In [ ]:
# suniv.head()

,IDLink,SentimentTitle,SentimentHeadline
0,tFrqIR6Chj,0.012229,0.009446
1,DVAaGErjlF,0.002380,0.037517
2,OT9UIZm5M2,-0.000316,-0.027790
3,lflGp3q2Fj,-0.010996,-0.042243
4,zDYG0SoovZ,-0.049601,-0.032183


In [ ]:
# meanz=pd.DataFrame(columns=['IDLink','SentimentTitle','SentimentHeadline'])
# meanz['IDLink'] = testtemp['IDLink']
# meanz['SentimentTitle'] = 0.6*sub['SentimentTitle']+0.4*suniv['SentimentTitle']
# meanz['SentimentHeadline'] = 0.6*sub['SentimentHeadline']+0.4*suniv['SentimentHeadline']

In [ ]:
# meanz.to_csv('submission4.csv',index=False)